# __Scraping Toronto Postal Code Data__

***
# **Part 1**
***

In [1]:
# !conda install -c conda-forge geocoder --yes

In [2]:
# Bringing in the necessary libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # import geocoder

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

import urllib.request as urlreq

from bs4 import BeautifulSoup as beaut

print('Libraries imported.')

Libraries imported.


In [5]:
# Using Beautiful Soup and urllib to scrape Wikipedia and isolate the table
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050"

# Pulling the data from the website
page = urlreq.urlopen(url)

# Formatting the data using an HTML parser
soup = beaut(page, 'html.parser')

# Isolating the Toronto data that we need
table = soup.find('table', class_='wikitable sortable')

# Checking the result
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North

In [5]:
# Creating lists from HTML formatted data
A=[]
B=[]
C=[]

#Looping through the HTML to scrape the essential data
for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [6]:
# Formatting the dataframe
# Creating the dataframe with the first 'PostalCode' column
tor = pd.DataFrame(A, columns=['PostalCode'])
tor['Borough'] = B # Appending the 'Borough'
tor['Neighborhood'] = C # Appending the 'Neighborhood'

# Filtering out 'Not assigned' data
tor1 = tor[tor['Borough'] != 'Not assigned'].reset_index(drop=True)

# Deleting some extra cahracters
tor1['Neighborhood'] = tor1['Neighborhood'].str.replace('\n', '')

# Sorting the table
tor1.sort_values(by=['PostalCode', 'Borough', 'Neighborhood'], inplace=True)

# Grouping the data by PostalCode and Borough
tor2 = tor1.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()

# Checking our final result
tor2.head(20)

PostalCode      Borough                                       Neighborhood
0         M1B  Scarborough                                     Malvern, Rouge
1         M1C  Scarborough             Highland Creek, Port Union, Rouge Hill
2         M1E  Scarborough                  Guildwood, Morningside, West Hill
3         M1G  Scarborough                                             Woburn
4         M1H  Scarborough                                          Cedarbrae
5         M1J  Scarborough                                Scarborough Village
6         M1K  Scarborough        East Birchmount Park, Ionview, Kennedy Park
7         M1L  Scarborough                    Clairlea, Golden Mile, Oakridge
8         M1M  Scarborough    Cliffcrest, Cliffside, Scarborough Village West
9         M1N  Scarborough                        Birch Cliff, Cliffside West
10        M1P  Scarborough  Dorset Park, Scarborough Town Centre, Wexford ...
11        M1R  Scarborough                                  Maryvale, Wexford
12        M1S  Scarborough                                          Agincourt
13        M1T  Scarborough            Clarks Corners, Sullivan, Tam O'Shanter
14        M1V  Scarborough  Agincourt North, L'Amoreaux East, Milliken, St...
15        M1W  Scarborough                                    L'Amoreaux West
16        M1X  Scarborough                                        Upper Rouge
17        M2H   North York                                  Hillcrest Village
18        M2J   North York                       Fairview, Henry Farm, Oriole
19        M2K   North York                                    Bayview Village

In [7]:
# Checking the shape
tor2.shape

(103, 3)

***
# **Part 2**
***

In [8]:
# Initializing our variables to None
#lat_lng_coords = None
#latitude = []
#longitude = []

# Looping until we get the coordinates (This method didn't work for me)
#for index, row in tor2[['PostalCode']].itertuples():
#    print(row)
#    while(lat_lng_coords is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(row))
#        lat_lng_coords = g.latlng
#    latitude.append(lat_lng_coords[0])
#    longitude.append(lat_lng_coords[1])

In [9]:
# Importing the coordinates csv
coords = pd.read_csv("Geospatial_Coordinates.csv")

# Renaming the column so that it matches our original table
coords.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

# Merging the two tables
tor3 = pd.merge(tor2, coords, on='PostalCode', how='left')

# Checking the final result
print(tor3.shape)
tor3

(103, 5)


PostalCode           Borough  \
0          M1B       Scarborough   
1          M1C       Scarborough   
2          M1E       Scarborough   
3          M1G       Scarborough   
4          M1H       Scarborough   
5          M1J       Scarborough   
6          M1K       Scarborough   
7          M1L       Scarborough   
8          M1M       Scarborough   
9          M1N       Scarborough   
10         M1P       Scarborough   
11         M1R       Scarborough   
12         M1S       Scarborough   
13         M1T       Scarborough   
14         M1V       Scarborough   
15         M1W       Scarborough   
16         M1X       Scarborough   
17         M2H        North York   
18         M2J        North York   
19         M2K        North York   
20         M2L        North York   
21         M2M        North York   
22         M2N        North York   
23         M2P        North York   
24         M2R        North York   
25         M3A        North York   
26         M3B        North York   
27         M3C        North York   
28         M3H        North York   
29         M3J        North York   
30         M3K        North York   
31         M3L        North York   
32         M3M        North York   
33         M3N        North York   
34         M4A        North York   
35         M4B         East York   
36         M4C         East York   
37         M4E      East Toronto   
38         M4G         East York   
39         M4H         East York   
40         M4J         East York   
41         M4K      East Toronto   
42         M4L      East Toronto   
43         M4M      East Toronto   
44         M4N   Central Toronto   
45         M4P   Central Toronto   
46         M4R   Central Toronto   
47         M4S   Central Toronto   
48         M4T   Central Toronto   
49         M4V   Central Toronto   
50         M4W  Downtown Toronto   
51         M4X  Downtown Toronto   
52         M4Y  Downtown Toronto   
53         M5A  Downtown Toronto   
54         M5B  Downtown Toronto   
55         M5C  Downtown Toronto   
56         M5E  Downtown Toronto   
57         M5G  Downtown Toronto   
58         M5H  Downtown Toronto   
59         M5J  Downtown Toronto   
60         M5K  Downtown Toronto   
61         M5L  Downtown Toronto   
62         M5M        North York   
63         M5N   Central Toronto   
64         M5P   Central Toronto   
65         M5R   Central Toronto   
66         M5S  Downtown Toronto   
67         M5T  Downtown Toronto   
68         M5V  Downtown Toronto   
69         M5W  Downtown Toronto   
70         M5X  Downtown Toronto   
71         M6A        North York   
72         M6B        North York   
73         M6C              York   
74         M6E              York   
75         M6G  Downtown Toronto   
76         M6H      West Toronto   
77         M6J      West Toronto   
78         M6K      West Toronto   
79         M6L        North York   
80         M6M              York   
81         M6N              York   
82         M6P      West Toronto   
83         M6R      West Toronto   
84         M6S      West Toronto   
85         M7A  Downtown Toronto   
86         M7R       Mississauga   
87         M7Y      East Toronto   
88         M8V         Etobicoke   
89         M8W         Etobicoke   
90         M8X         Etobicoke   
91         M8Y         Etobicoke   
92         M8Z         Etobicoke   
93         M9A         Etobicoke   
94         M9B         Etobicoke   
95         M9C         Etobicoke   
96         M9L        North York   
97         M9M        North York   
98         M9N              York   
99         M9P         Etobicoke   
100        M9R         Etobicoke   
101        M9V         Etobicoke   
102        M9W         Etobicoke   

                                          Neighborhood   Latitude  Longitude  
0                                       Malvern, Rouge  43.806686 -79.194353  
1               Highland Creek, Port Union, Rouge Hill  43.784535 -79.160497  
2                    G

***
# **Part 3**
***

In [10]:
# Create map of Toronto using latitude and longitude values
latitude = 43.70
longitude = -79.35

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(tor3['Latitude'], tor3['Longitude'], tor3['Borough'], tor3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [11]:
CLIENT_ID = '0CF1WVVT3IUF4XDKK3RQDJNWB5NTQIKT1YA2YZ5DGJMRPIIJ' # your Foursquare ID
CLIENT_SECRET = 'TMT0GGJNQJZBF1HG2WDHKSIYA4LGAC55GWHBKDFRDZKON0PZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
tor_venues = getNearbyVenues(names=tor3['Neighborhood'],
                                   latitudes=tor3['Latitude'],
                                   longitudes=tor3['Longitude']
                                  )
print(tor_venues.shape)
tor_venues.head()

Malvern, Rouge
Highland Creek, Port Union, Rouge Hill
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South, Flemingdon Park
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
Riverdale, The Danf

Neighborhood  Neighborhood Latitude  \
0                          Malvern, Rouge              43.806686   
1  Highland Creek, Port Union, Rouge Hill              43.784535   
2  Highland Creek, Port Union, Rouge Hill              43.784535   
3       Guildwood, Morningside, West Hill              43.763573   
4       Guildwood, Morningside, West Hill              43.763573   

   Neighborhood Longitude                           Venue  Venue Latitude  \
0              -79.194353                         Wendy’s       43.807448   
1              -79.160497           Royal Canadian Legion       43.782533   
2              -79.160497  Scarborough Historical Society       43.788755   
3              -79.188711               G & G Electronics       43.765309   
4              -79.188711                      Marina Spa       43.766000   

   Venue Longitude        Venue Category  
0       -79.199056  Fast Food Restaurant  
1       -79.163085                   Bar  
2       -79.162438        History Museum  
3       -79.191537     Electronics Store  
4       -79.191000                   Spa

In [14]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()

Yoga Studio  Accessories Store  Afghan Restaurant  Airport  \
0            0                  0                  0        0   
1            0                  0                  0        0   
2            0                  0                  0        0   
3            0                  0                  0        0   
4            0                  0                  0        0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                   0             0               0                0   
1                   0             0               0                0   
2                   0             0               0                0   
3                   0             0               0                0   
4                   0             0               0                0   

   Airport Terminal  American Restaurant  Antique Shop  Aquarium  Art Gallery  \
0                 0                    0             0         0            0   
1                 0                    0             0         0            0   
2                 0                    0             0         0            0   
3                 0                    0             0         0            0   
4                 0                    0             0         0            0   

   Art Museum  Arts & Crafts Store  Asian Restaurant  Athletics & Sports  \
0           0                    0                 0                   0   
1           0                    0                 0                   0   
2           0                    0                 0                   0   
3           0                    0                 0                   0   
4           0                    0                 0                   0   

   Auto Garage  Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  Bakery  \
0            0              0          0           0           0       0   
1            0              0          0           0           0       0   
2            0              0          0           0           0       0   
3            0              0          0           0           0       0   
4            0              0          0           0           0       0   

   Bank  Bar  Baseball Field  Baseball Stadium  Basketball Court  \
0     0    0               0                 0                 0   
1     0    1               0                 0                 0   
2     0    0               0                 0                 0   
3     0    0               0                 0                 0   
4     0    0               0                 0                 0   

   Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  Beer Store  \
0                   0      0                0         0           0   
1                   0      0                0         0           0   
2                   0      0                0         0           0   
3                   0      0                0         0           0   
4                   0      0                0         0           0   

   Belgian Restaurant  Bike Shop  Bistro  Boat or Ferry  Bookstore  Boutique  \
0                   0          0       0              0          0         0   
1                   0          0       0              0          0         0   
2                   0          0       0              0          0         0   
3                   0          0       0              0          0         0   
4                   0          0       0              0          0         0   

   Brazilian Restaurant  Breakfast Spot  Brewery  Bridal Shop  \
0                     0               0        0            0   
1                     0               0        0            0   
2                     0               0        0            0   
3                     0               0        0            0   
4                     0               0        0            0   

   Bubble Tea Shop  Building  Burger Joint  Burrito Place  Bus Line  \

In [15]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped.head()

Neighborhood  Yoga Studio  \
0                           Adelaide, King, Richmond          0.0   
1                                          Agincourt          0.0   
2  Agincourt North, L'Amoreaux East, Milliken, St...          0.0   
3  Albion Gardens, Beaumond Heights, Humbergate, ...          0.0   
4                             Alderwood, Long Branch          0.0   

   Accessories Store  Afghan Restaurant  Airport  Airport Food Court  \
0                0.0                0.0      0.0                 0.0   
1                0.0                0.0      0.0                 0.0   
2                0.0                0.0      0.0                 0.0   
3                0.0                0.0      0.0                 0.0   
4                0.0                0.0      0.0                 0.0   

   Airport Gate  Airport Lounge  Airport Service  Airport Terminal  \
0           0.0             0.0              0.0               0.0   
1           0.0             0.0              0.0               0.0   
2           0.0             0.0              0.0               0.0   
3           0.0             0.0              0.0               0.0   
4           0.0             0.0              0.0               0.0   

   American Restaurant  Antique Shop  Aquarium  Art Gallery  Art Museum  \
0                 0.02           0.0       0.0         0.01        0.01   
1                 0.00           0.0       0.0         0.00        0.00   
2                 0.00           0.0       0.0         0.00        0.00   
3                 0.00           0.0       0.0         0.00        0.00   
4                 0.00           0.0       0.0         0.00        0.00   

   Arts & Crafts Store  Asian Restaurant  Athletics & Sports  Auto Garage  \
0                  0.0              0.02            0.000000          0.0   
1                  0.0              0.00            0.000000          0.0   
2                  0.0              0.00            0.000000          0.0   
3                  0.0              0.00            0.000000          0.0   
4                  0.0              0.00            0.090909          0.0   

   Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank   Bar  \
0            0.0        0.0         0.0         0.0    0.02   0.0  0.03   
1            0.0        0.0         0.0         0.0    0.00   0.0  0.00   
2            0.0        0.0         0.0         0.0    0.00   0.0  0.00   
3            0.0        0.0         0.0         0.0    0.00   0.0  0.00   
4            0.0        0.0         0.0         0.0    0.00   0.0  0.00   

   Baseball Field  Baseball Stadium  Basketball Court  Basketball Stadium  \
0             0.0               0.0               0.0                 0.0   
1             0.0               0.0               0.0                 0.0   
2             0.0               0.0               0.0                 0.0   
3             0.0               0.0               0.0                 0.0   
4             0.0               0.0               0.0                 0.0   

   Beach  Bed & Breakfast  Beer Bar  Beer Store  Belgian Restaurant  \
0    0.0              0.0       0.0    0.000000                 0.0   
1    0.0              0.0       0.0    0.000000                 0.0   
2    0.0              0.0       0.0    0.000000                 0.0   
3    0.0              0.0       0.0    0.111111                 0.0   
4    0.0              0.0       0.0    0.000000                 0.0   

   Bike Shop  Bistro  Boat or Ferry  Bookstore  Boutique  \
0        0.0     0.0            0.0       0.02       0.0   
1        0.0     0.0            0.0       0.00       0.0   
2        0.0     0.0            0.0       0.00       0.0   
3        0.0     0.0            0.0       0.00       0.0   
4        0.0     0.0            0.0       0.00       0.0   

   Brazilian Restaurant  Breakfast Spot  Brewery  Bridal Shop  \
0                  0.01            0.02      0.0          0.0   
1                  0.00  

In [21]:
num_top_venues = 5

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
#    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
#    print('\n')

----Adelaide, King, Richmond----
----Agincourt----
----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
----Alderwood, Long Branch----
----Bathurst Manor, Downsview North, Wilson Heights----
----Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara----
----Bayview Village----
----Bedford Park, Lawrence Manor East----
----Berczy Park----
----Birch Cliff, Cliffside West----
----Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe----
----Brockton, Exhibition Place, Parkdale Village----
----Business Reply Mail Processing Centre 969 Eastern----
----CFB Toronto, Downsview East----
----Cabbagetown, St. James Town----
----Caledonia-Fairbanks----
----Canada Post Gateway Processing Centre----
----Cedarbrae----
----Central Bay Street----
----Chinatown, Grange Park, Kensington Market----
----Christie----
---

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                           Adelaide, King, Richmond           Coffee Shop   
1                                          Agincourt        Breakfast Spot   
2  Agincourt North, L'Amoreaux East, Milliken, St...            Playground   
3  Albion Gardens, Beaumond Heights, Humbergate, ...         Grocery Store   
4                             Alderwood, Long Branch           Pizza Place   

       2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                 Restaurant                  Café       Thai Restaurant   
1  Latin American Restaurant        Clothing Store          Skating Rink   
2                       Park         Women's Store      Doner Restaurant   
3                Pizza Place              Pharmacy        Sandwich Place   
4                   Pharmacy        Sandwich Place                   Gym   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0            Steakhouse                   Bar                   Gym   
1                Lounge     Electronics Store   Empanada Restaurant   
2          Dessert Shop    Dim Sum Restaurant                 Diner   
3            Beer Store  Fast Food Restaurant   Fried Chicken Joint   
4           Coffee Shop          Skating Rink    Athletics & Sports   

         8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0                    Gastropub                Lounge     Seafood Restaurant  
1  Eastern European Restaurant   Dumpling Restaurant     Dim Sum Restaurant  
2               Discount Store   Distribution Center                Dog Run  
3      Comfort Food Restaurant          Dessert Shop    Dumpling Restaurant  
4                 Dance Studio                   Pub                   Pool

In [24]:
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [26]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = tor3

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# Convert floats to ints, necessary later in the code
tor_merged['Cluster Labels'] = tor_merged['Cluster Labels'].fillna(0.0).astype(int)

tor_merged.head() # check the last columns!

PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Malvern, Rouge  43.806686   
1        M1C  Scarborough  Highland Creek, Port Union, Rouge Hill  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.194353               2  Fast Food Restaurant         Women's Store   
1 -79.160497               0        History Museum                   Bar   
2 -79.188711               0     Electronics Store                   Spa   
3 -79.216917               0           Coffee Shop     Korean Restaurant   
4 -79.239476               0      Hakka Restaurant                Bakery   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0            Donut Shop    Dim Sum Restaurant                 Diner   
1         Women's Store      Doner Restaurant    Dim Sum Restaurant   
2   Rental Car Location          Intersection        Medical Center   
3         Women's Store      Doner Restaurant    Dim Sum Restaurant   
4    Athletics & Sports                  Bank       Thai Restaurant   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0        Discount Store   Distribution Center               Dog Run   
1                 Diner        Discount Store   Distribution Center   
2                  Bank    Mexican Restaurant   Distribution Center   
3                 Diner        Discount Store   Distribution Center   
4  Caribbean Restaurant           Gas Station   Fried Chicken Joint   

  9th Most Common Venue 10th Most Common Venue  
0      Doner Restaurant              Drugstore  
1               Dog Run             Donut Shop  
2                 Diner         Discount Store  
3               Dog Run              Drugstore  
4         Women's Store         Discount Store

In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters